<a href="https://colab.research.google.com/github/chloejeannin/YOLOBee/blob/main/training_06/training_x/tracking_bees_06_additional_frames_x.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Introduction:
Strating over the training with new augmentations, especially RandomBrightnessContrast, which is very important in our case because of the shadows. The training was not performed on Google Collab but on the ondemand.metacentrum.cz. The training was made with the model yolov5x. Because the Yolov5x is bigger than the other model, the training is using 2 GPUs for not to take too much time

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
%cd /content/gdrive/Shareddrives/YOLObee_drive/02_Code/YOLOv5/

/content/gdrive/Shareddrives/YOLObee_drive/02_Code/YOLOv5


In [ ]:
!ls

datasets    train_data	   train_data_03  train_data1  train_data_part1  yolov5
SciAugment  train_data_02  train_data_04  train_data2  train_data.zip


Augmentation
---

In [ ]:
#! git clone https://github.com/martinschatz-cz/SciAugment.git

In [ ]:
!pip install -r /content/gdrive/Shareddrives/YOLObee_drive/02_Code/YOLOv5/SciAugment/requirements.txt -v

Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.7/14.7 MB 50.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 51.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.5/313.5 kB 21.5 MB/s eta 0:00:00
  Attempting uninstall: imageio
    Found existing installation: imageio 2.31.6
    Uninstalling imageio-2.31.6:
      Removing file or directory /usr/local/bin/imageio_download_bin
      Removing file or directory /usr/local/bin/imageio_remove_bin
      Removing file or directory /usr/local/lib/python3.10/dist-packages/imageio-2.31.6.dist-info/
      Removing file or directory /usr/local/lib/python3.10/dist-packages/imageio/
      Successfully uninstalled imageio-2.31.6
  changing mode of /usr/local/bin/imageio_download_bin to 755
  changing mode of /usr/local/bin/imageio_remove_bin to

In [ ]:
!pip install watermark

%load_ext watermark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 15.2 MB/s eta 0:00:00


In [ ]:
%watermark -v -p albumentations,opencv-python-headless,imgaug,cv2,yolov5,torch

Python implementation: CPython
Python version       : 3.10.12
IPython version      : 7.34.0

albumentations        : 1.4.8
opencv-python-headless: not installed
imgaug                : 0.4.0
cv2                   : 4.8.0
yolov5                : unknown
torch                 : 2.3.0+cu121



In [ ]:
# @title
# SciAug_tools
import copy
import glob
import math
import os
import random
from shutil import rmtree
from sys import stdout
from xml.dom import minidom

import albumentations as A
import cv2
import imgaug as ia
import imgaug.augmenters as iaa
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage


class SciAugment:
    version = "0.2.0"
    source = "https://github.com/martinschatz-cz/SciAugment"
    author = "Martin Schätz"
    aug_type = ""
    augment = []
    channel_augment = []
    aug_dict = {
        -1: "no augmentation",
        0: "HorizontalFlip(p=1)",
        1: "RandomBrightnessContrast(contrast_limit=0.2,p=1)",
        2: "MultiplicativeNoise(multiplier=0.5, p=1)",
        3: "RandomSizedBBoxSafeCrop(250, 250, erosion_rate=0.0, interpolation=1, p=1.0)",
        4: "Blur(blur_limit=(10, 10), p=0)",
        5: "Transpose(1)",
        6: "RandomRotate90(p=1)",
        7: "ShiftScaleRotate(p=1)",
        8: "VerticalFlip(p=1)",
        9: "RandomBrightnessContrast(brightness_limit=0.2,p=1)",
    }

    aug_channel_dict = {
        -1: "no augmentation",
        0: "RandomBrightnessContrast(contrast_limit=0.2,p=1)",
        1: "MultiplicativeNoise(multiplier=0.5, p=1)",
        2: "Blur(blur_limit=(10, 10), p=0)",
        3: "RandomBrightnessContrast(brightness_limit=0.2,p=1)",
        4: "Superpixels (p_replace=0.1, n_segments=20, max_size=64, interpolation=1, p=1)",
        5: "GaussNoise (var_limit=(10.0, 50.0), mean=0, p=1)",
    }

    #######general functions###########
    def __new__(cls, *args, **kwargs):
        print("New instance of SciAugment.")
        return super().__new__(cls)

    def __init__(self, aug_type: str = "Default", channel_aug: bool = False):
        self.augmet_type = aug_type
        print("Selected augmentation type: {}".format(self.augmet_type))
        if aug_type == "Default":
            self.augment = [-1, 0, 3, 5, 6, 7, 8]
            if channel_aug:
                self.channel_augment = range(-1, 5, 1)

        if aug_type == "fluorescece_microscopy":
            self.augment = range(-1, 9, 1)
            if channel_aug:
                self.channel_augment = [-1, 1, 2, 3, 4, 5]

        if aug_type == "all":
            self.augment = range(-1, 9, 1)
            if channel_aug:
                self.channel_augment = range(-1, 5, 1)

        if aug_type == "no_augment":
            self.augment = [-1]
            print(
                "No augment setting will only divide images and labels to train_data folder."
            )
            self.channel_augment = []

        print("\n")
        self.explain()

    def explain(self):
        print("Version: {}".format(self.version))
        print("\n")
        print("Selected augmentation:")
        for aug in self.augment:
            print(self.aug_dict[aug])

        if self.channel_augment:
            print("\n")
            print("Selected channel wise augmentation:")
            for ch_aug in self.channel_augment:
                print(self.aug_channel_dict[ch_aug])

    def info(self):
        print("Version: {}".format(self.version))
        print("\n")
        print("Selected augmentation:")
        for aug in self.augment:
            print(self.aug_dict[aug])

        if self.channel_augment:
            print("\n")
            print("Selected channel wise augmentation:")
            for ch_aug in self.channel_augment:
                print(self.aug_channel_dict[ch_aug])

        print("\n")
        print("\n")
        print("Source: {}".format(self.source))
        print("Author: {}".format(self.author))

    def read_image(self, images_path: str, filename: str):
        """
        Uses OpenCV to read RGB images
        :param images_path (str):
        :param filename (str):
        :return: RGB image
        """

        # OpenCV uses BGR channels
        img = cv2.imread(images_path + filename)
        return img

    def read_image_unchanged(self, images_path: str, filename: str):
        """
        Uses OpenCV to read RGBA or 4 channel (maximum) images
        :param images_path (str):
        :param filename (str):
        :return: RGB image
        """
        # https://learnopencv.com/read-an-image-in-opencv-python-cpp/
        # OpenCV uses BGR channels
        img = cv2.imread(
            images_path + filename, cv2.IMREAD_UNCHANGED | cv2.IMREAD_ANYDEPTH
        )
        return img

    def read_yolo(self, filename: str) -> float:
        """
        Reads YOLO type of object anotation for albumentation to read
        :param filename (str):
        :return yolo_coords:
        """
        yolo_coords = []
        with open(filename, "r") as fname:
            for yolo in fname:
                x = yolo.strip().split(" ")
                x.append(x[0])
                x.pop(0)
                x[0] = float(x[0])
                x[1] = float(x[1])
                x[2] = float(x[2])
                x[3] = float(x[3])
                # print(x)
                yolo_coords.append(x)
        return yolo_coords

    def write_yolo(self, coords: float, name: str):
        """
        Write YOLO type object coordinates to txt file
        :param coords (float): list of coordinates
        :param name (str): name of file to write to
        :return:
        """
        with open(name + ".txt", "w") as f:
            for x in coords:
                f.write("%s %s %s %s %s \n" % (x[-1], x[0], x[1], x[2], x[3]))
        return 0

    def create_train_data_folder(self):
        try:
            dir = "train_data"
            if os.path.exists(dir):
                print("Train_data folder already exists!")  # , and will be removed.')
                return 1
                # shutil.rmtree(dir)
            else:
                os.mkdir(dir)

            # info_path = dir + '/info.txt'
            # with open(info_path, 'w') as f:
            #   f.writelines(self.info())

            dir_image_train = "train_data/images/train"
            dir_image_val = "train_data/images/val"
            dir_label_train = "train_data/labels/train"
            dir_label_val = "train_data/labels/val"

            os.mkdir("train_data/images")
            os.mkdir("train_data/labels")
            os.mkdir(dir_image_train)
            os.mkdir(dir_image_val)
            os.mkdir(dir_label_train)
            os.mkdir(dir_label_val)

            return 0
        except:
            print("Error while creating new train_data folder structure!")
            return 1

    #########transform functions########
    def get_transform(loop: int):
        """
      !!!OLD FUNCTION, DO NOT USE!!!
              Generating image an label augmentation for YOLO with name tag of augmentations.

              :param loop (int): Number of augmentation from 0 to 8

              :return:
              A: transform function with BboxParams set for Yolo
              name (string): Name where relevant to position is bit length 11 for:
                1:Shift
                2:Scale
                3:Rotate
                4:VerticalFlip
                5:HorizontalFlip
                6:RandomContrast
                7:MultiplicativeNoise(multiplier=0.5, p=1)
                8:RandomSizedBBoxSafeCrop (250, 250, erosion_rate=0.0, interpolation=1, p=1.0)
                9:Blur(blur_limit=(10, 10), p=0)
                10:Transpose
                11:RandomRotate90
                12:RandomBrightness
        """
        if loop == 0:
            transform = A.Compose(
                [
                    A.HorizontalFlip(p=1),
                ],
                bbox_params=A.BboxParams(format="yolo"),
            )
            name = "00001000000"
        elif loop == 1:
            transform = A.Compose(
                [
                    A.RandomBrightnessContrast(p=1),
                ],
                bbox_params=A.BboxParams(format="yolo"),
            )
            name = "00000100000"
        elif loop == 2:
            transform = A.Compose(
                [
                    A.HorizontalFlip(p=0),
                    A.MultiplicativeNoise(multiplier=0.5, p=1),
                ],
                bbox_params=A.BboxParams(format="yolo"),
            )
            name = "00000010000"
        elif loop == 3:
            transform = A.Compose(
                [
                    # A.CenterCrop(width=250, height=250, p=1)
                    A.RandomSizedBBoxSafeCrop(
                        250, 250, erosion_rate=0.0, interpolation=1, p=1.0
                    )
                ],
                bbox_params=A.BboxParams(format="yolo"),
            )
            name = "00000001000"
        elif loop == 4:
            transform = A.Compose(
                [A.Blur(blur_limit=(10, 10), p=1)],
                bbox_params=A.BboxParams(format="yolo"),
            )
            name = "00000000100"
        elif loop == 5:
            transform = A.Compose(
                [A.Transpose(p=11)], bbox_params=A.BboxParams(format="yolo")
            )
            name = "00000000010"
        elif loop == 6:
            transform = A.Compose(
                [A.RandomRotate90(p=1)], bbox_params=A.BboxParams(format="yolo")
            )
            name = "00000000001"
        elif loop == 7:
            transform = A.Compose(
                [A.ShiftScaleRotate(p=1)], bbox_params=A.BboxParams(format="yolo")
            )
            name = "11100000000"
        elif loop == 8:
            transform = A.Compose(
                [A.VerticalFlip(p=1)], bbox_params=A.BboxParams(format="yolo")
            )
            name = "00010000000"
        # elif loop == 7:
        #     transform = A.Compose([
        #         A.ImageCompression(quality_lower=0, quality_upper=1, p=0.2)
        #     ], bbox_params=A.BboxParams(format='yolo'))
        # elif loop == 8:
        #     transform = A.Compose([
        #         A.CoarseDropout(max_holes=50, max_height=40,
        #                  max_width=40, fill_value=128, p=0)
        #     ], bbox_params=A.BboxParams(format='pascal_voc'))

        # transform = A.Compose([
        #         A.HorizontalFlip(p=1),
        #     ], bbox_params=A.BboxParams(format='yolo'))

        return transform, name

    def _h_flip():
        transform = A.Compose(
            [
                A.HorizontalFlip(p=1),
            ],
            bbox_params=A.BboxParams(format="yolo"),
        )
        name = "000010000000"
        return transform, name

    def _rand_contrast():
        transform = A.Compose(
            [
                A.RandomBrightnessContrast(contrast_limit=0.2, p=1),
            ],
            bbox_params=A.BboxParams(format="yolo"),
        )
        name = "000001000000"
        return transform, name

    def _multi_noise():
        transform = A.Compose(
            [
                A.HorizontalFlip(p=0),
                A.MultiplicativeNoise(multiplier=0.5, p=1),
            ],
            bbox_params=A.BboxParams(format="yolo"),
        )
        name = "000000100000"
        return transform, name

    def _rand_size_crop():
        transform = A.Compose(
            [
                # A.CenterCrop(width=250, height=250, p=1)
                A.RandomSizedBBoxSafeCrop(
                    250, 250, erosion_rate=0.0, interpolation=1, p=1.0
                )
            ],
            bbox_params=A.BboxParams(format="yolo"),
        )
        name = "000000010000"
        return transform, name

    def _im_blur():
        transform = A.Compose(
            [A.Blur(blur_limit=(10, 10), p=1)], bbox_params=A.BboxParams(format="yolo")
        )
        name = "000000001000"
        return transform, name

    def _im_transpose():
        transform = A.Compose(
            [A.Transpose(p=1)], bbox_params=A.BboxParams(format="yolo")
        )
        name = "000000000100"
        return transform, name

    def _rand_rotate():
        transform = A.Compose(
            [A.RandomRotate90(p=1)], bbox_params=A.BboxParams(format="yolo")
        )
        name = "000000000010"
        return transform, name

    def _shift_scale_rotate():
        transform = A.Compose(
            [A.ShiftScaleRotate(p=1)], bbox_params=A.BboxParams(format="yolo")
        )
        name = "111000000000"
        return transform, name

    def _v_flip():
        transform = A.Compose(
            [A.VerticalFlip(p=1)], bbox_params=A.BboxParams(format="yolo")
        )
        name = "000100000000"
        return transform, name

    def _no_augment():
        transform = A.Compose(
            [A.VerticalFlip(p=0)], bbox_params=A.BboxParams(format="yolo")
        )
        name = "000000000000"
        return transform, name

    def _rand_brightness():
        transform = A.Compose(
            [
                A.RandomBrightnessContrast(brightness_limit=0.2, p=1),
            ],
            bbox_params=A.BboxParams(format="yolo"),
        )
        name = "000000000001"
        return transform, name

    # for 16 bit A.ToFloat(max_value=65535.0),
    #     """
    #       1:Shift
    #       2:Scale
    #       3:Rotate
    #       4:VerticalFlip
    #       5:HorizontalFlip
    #       6:RandomBrightnessContrast
    #       7:MultiplicativeNoise(multiplier=0.5, p=1)
    #       8:RandomSizedBBoxSafeCrop (250, 250, erosion_rate=0.0, interpolation=1, p=1.0)
    #       9:Blur(blur_limit=(10, 10), p=0)
    #       10:Transpose
    #       11:RandomRotate90
    #     """

    aug_functions = {
        -1: _no_augment,
        0: _h_flip,
        1: _rand_contrast,
        2: _multi_noise,
        3: _rand_size_crop,
        4: _im_blur,
        5: _im_transpose,
        6: _rand_rotate,
        7: _shift_scale_rotate,
        8: _v_flip,
        9: _rand_brightness,
    }

    # add functions
    def _no_augment_ch():
        transform = A.Compose([A.VerticalFlip(p=0)])
        name = "-NA"
        return transform, name

    def _rand_contrast_ch():
        transform = A.Compose([A.RandomBrightnessContrast(contrast_limit=0.2, p=1)])
        name = "-RC"
        return transform, name

    def _multi_noise_ch():
        transform = A.Compose([A.MultiplicativeNoise(multiplier=0.5, p=1)])
        name = "-MN"
        return transform, name

    def _im_blur_ch():
        transform = A.Compose([A.Blur(blur_limit=(10, 10), p=1)])
        name = "-B"
        return transform, name

    def _rand_brightness_ch():
        transform = A.Compose([A.RandomBrightnessContrast(brightness_limit=0.2, p=1)])
        name = "-RB"
        return transform, name

    def _superpixels_ch():
        transform = A.Compose(
            [
                A.Superpixels(
                    p_replace=0.1, n_segments=20, max_size=64, interpolation=1, p=1
                )
            ]
        )
        name = "-SP"
        return transform, name

    def _gauss_noise_ch():
        transform = A.Compose([A.GaussNoise(var_limit=(10.0, 50.0), mean=0, p=1)])
        name = "-GN"
        return transform, name

    channel_aug_functions = {
        -1: _no_augment_ch,
        0: _rand_contrast_ch,
        1: _multi_noise_ch,
        2: _im_blur_ch,
        3: _rand_brightness_ch,
        4: _superpixels_ch,
        5: _gauss_noise_ch,
    }

    #####apply functions######
    def augment_data(
        self,
        images_path: str,
        train: float = 0.7,
        image_format: str = ".png",
        output_image_format: str = ".jpg",
    ):
        """
        Augment input images and YOLO files as defined in get_transform() function and save then in train_dir prepared for training and test/val based on 70/30 rule.
        :param images_path (str):
        :param train (float): train/val parameter for generating training data set, default 70% train 30% val
        :param image_format (str): image format name ('.png' default)
        :return:
        """
        count = 0
        # dir = 'train_data'

        dir_image_train = "train_data/images/train"
        dir_image_val = "train_data/images/val"
        dir_label_train = "train_data/labels/train"
        dir_label_val = "train_data/labels/val"

        folder_err = self.create_train_data_folder()
        if folder_err:
            print("Please remove existing trin_data folder and try again.")
            return 1

        # image_format = ".png"
        # train = 0.7
        test = 1 - train
        files_to_process = sorted(os.listdir(images_path))
        print("Num of files: " + str(len(files_to_process)))

        #define empty title, ext, xmlTitle, txtExt
        title = 'title'
        ext = 'ext'
        xmlTitle = 'xmlTitle'
        txtExt = 'txtExt'

        for filename in files_to_process:
            print("Processing: " + filename)

            if filename.endswith(image_format.lower()) or filename.endswith(
                image_format.upper()
            ):
                title, ext = os.path.splitext(os.path.basename(filename))
                print(images_path)
                print(filename)
                image = self.read_image(images_path, filename)
            if filename.endswith(".txt"):
                xmlTitle, txtExt = os.path.splitext(os.path.basename(filename))
                if xmlTitle == title:
                    # bboxes = getCoordinates(filename)
                    bboxes = self.read_yolo(images_path + xmlTitle + ".txt")
                    print(images_path + xmlTitle + ".txt")
                    for aug in self.augment:
                        img = copy.deepcopy(image)
                        transform, name_tag = self.aug_functions[aug]()
                        dice = random.uniform(0, 1)
                        # try:
                        transformed = transform(image=img, bboxes=bboxes)
                        transformed_image = transformed["image"]
                        transformed_bboxes = transformed["bboxes"]
                        name = (
                            title
                            + "_"
                            + str(count)
                            + "_"
                            + name_tag
                            + output_image_format
                        )
                        # print(name)
                        if dice <= train:
                            p_name = dir_image_train + "/" + name
                        else:
                            p_name = dir_image_val + "/" + name

                        cv2.imwrite(p_name, transformed_image)
                        print("Writing " + name)
                        # print(transformed_bboxes)
                        # writeVoc(transformed_bboxes, count, transformed_image)
                        # pTitle='/content/'+dir+'/'+title
                        if dice <= train:
                            p_title = (
                                dir_label_train
                                + "/"
                                + title
                                + "_"
                                + str(count)
                                + "_"
                                + name_tag
                            )
                        else:
                            p_title = (
                                dir_label_val
                                + "/"
                                + title
                                + "_"
                                + str(count)
                                + "_"
                                + name_tag
                            )

                        self.write_yolo(transformed_bboxes, p_title)
                        count = count + 1
                        # except:
                        # print("Bounding Box exception!!!")
                        # pass
        print("Process created {} images".format(count))

    def augment_data_per_channel(
        self,
        images_path: str,
        train: float = 0.7,
        image_format: str = ".png",
        output_image_format: str = ".png",
    ):
        """
        Augment input images and YOLO files as defined in get_transform() function and save then in train_dir prepared for training and test/val based on 70/30 rule.
        :param images_path (str):
        :param train (float): train/val parameter for generating training data set, default 70% train 30% val
        :param image_format (str): image format name ('.png' default)
        :return:
        """
        count = 0
        # dir = 'train_data'

        dir_image_train = "train_data/images/train"
        dir_image_val = "train_data/images/val"
        dir_label_train = "train_data/labels/train"
        dir_label_val = "train_data/labels/val"

        folder_err = self.create_train_data_folder()
        if folder_err:
            print("Please remove existing trin_data folder and try again.")
            return 1

        # image_format = ".png"
        # train = 0.7
        test = 1 - train
        files_to_process = sorted(os.listdir(images_path))
        print("Num of files: " + str(len(files_to_process)))

        for filename in files_to_process:
            print("Processing: " + filename)

            if filename.endswith(image_format.lower()) or filename.endswith(
                image_format.upper()
            ):
                title, ext = os.path.splitext(os.path.basename(filename))
                print(images_path)
                print(filename)
                image = self.read_image_unchanged(images_path, filename)
            if filename.endswith(".txt"):
                xmlTitle, txtExt = os.path.splitext(os.path.basename(filename))
                if xmlTitle == title:
                    # bboxes = getCoordinates(filename)
                    bboxes = self.read_yolo(images_path + xmlTitle + ".txt")
                    print(images_path + xmlTitle + ".txt")
                    for aug in self.augment:
                        img = copy.deepcopy(image)
                        transform, name_tag = self.aug_functions[aug]()
                        dice = random.uniform(0, 1)
                        #####apply also channel augmentation#####
                        # try:
                        transformed = transform(image=img, bboxes=bboxes)
                        transformed_image = transformed["image"]
                        transformed_bboxes = transformed["bboxes"]

                        # go through channel transports
                        for ch_aug in self.channel_augment:

                            transform_ch, ch_name_tag = self.channel_aug_functions[
                                ch_aug
                            ]()

                            # split channels
                            im_split = cv2.split(transformed_image)

                            for ch in range(0, len(im_split), 1):
                                channel = im_split[ch]

                                transformed = transform_ch(image=channel)
                                transformed_channel = transformed["image"]

                                im_merge = copy.deepcopy(im_split)
                                im_merge[ch] = transformed_channel
                                merged_transformed = cv2.merge(im_merge)
                                name_tag_ch = "_ch-" + str(ch + 1) + ch_name_tag
                                # print(name_ch)
                                # cv2_imshow(merged)
                                # save
                                name = (
                                    title
                                    + "_"
                                    + str(count)
                                    + "_"
                                    + name_tag
                                    + name_tag_ch
                                )
                                # print(name)
                                if dice <= train:
                                    p_name = (
                                        dir_image_train
                                        + "/"
                                        + name
                                        + output_image_format
                                    )
                                else:
                                    p_name = (
                                        dir_image_val + "/" + name + output_image_format
                                    )

                                cv2.imwrite(p_name, merged_transformed)
                                print("Writing " + name)
                                # print(transformed_bboxes)
                                # writeVoc(transformed_bboxes, count, transformed_image)
                                # pTitle='/content/'+dir+'/'+title
                                if dice <= train:
                                    p_title = dir_label_train + "/" + name
                                else:
                                    p_title = dir_label_val + "/" + name

                                self.write_yolo(transformed_bboxes, p_title)
                                count = count + 1
                        # except:
                        # print("Augmentation exception!!!")
                        # pass
        print("Process created {} images".format(count))

In [ ]:
from SciAugment.utilities.SciAug_tools import *

In [ ]:
#@markdown Specifie a path to folder with images and YOLO anotations
input_images_folder = '/content/gdrive/Shareddrives/YOLObee_drive/01_Data/frames_04+frames_2/'  # @param{type: 'string'}
input_image_format = '.jpg'  # @param{type: 'string'}

In [ ]:
aug = SciAugment()

New instance of SciAugment.
Selected augmentation type: Default


Version: 0.1.0


Selected augmentation:
HorizontalFlip(p=1)
RandomSizedBBoxSafeCrop(250, 250, erosion_rate=0.0, interpolation=1, p=1.0)
Transpose(1)
RandomRotate90(p=1)
ShiftScaleRotate(p=1)
VerticalFlip(p=1)


In [ ]:
%ls

class1.yaml  SciAugment/     train_data_03/  train_data_part1/  yolov5/
datasets/    train_data/     train_data1/    train_data.zip
gdrive/      train_data_02/  train_data2/    train.zip


In [ ]:
%cd content/

/content


In [ ]:
%ls

gdrive/  sample_data/


In [ ]:
aug.augment_data(images_path=input_images_folder, image_format=input_image_format)

Num of files: 214
Processing: Copie de frame_0002.jpg
/content/gdrive/Shareddrives/YOLObee_drive/01_Data/frames_04+frames_2/
Copie de frame_0002.jpg
Processing: Copie de frame_0002.txt
/content/gdrive/Shareddrives/YOLObee_drive/01_Data/frames_04+frames_2/Copie de frame_0002.txt
Writing Copie de frame_0002_0_00001000000.jpg
Writing Copie de frame_0002_1_00000001000.jpg
Writing Copie de frame_0002_2_00000000010.jpg
Writing Copie de frame_0002_3_00000000001.jpg
Writing Copie de frame_0002_4_11100000000.jpg
Writing Copie de frame_0002_5_00010000000.jpg
Processing: Copie de frame_0003.jpg
/content/gdrive/Shareddrives/YOLObee_drive/01_Data/frames_04+frames_2/
Copie de frame_0003.jpg
Processing: Copie de frame_0003.txt
/content/gdrive/Shareddrives/YOLObee_drive/01_Data/frames_04+frames_2/Copie de frame_0003.txt
Writing Copie de frame_0003_6_00001000000.jpg
Writing Copie de frame_0003_7_00000001000.jpg
Writing Copie de frame_0003_8_00000000010.jpg
Writing Copie de frame_0003_9_00000000001.jpg


In [ ]:
import shutil
shutil.copytree('/content/train_data','gdrive/Shareddrives/YOLObee_drive/02_Code/YOLOv5/train_data')

'gdrive/Shareddrives/YOLObee_drive/02_Code/YOLOv5/train_data'

In [ ]:
import shutil
shutil.make_archive('gdrive/Shareddrives/YOLObee_drive/02_Code/train_data_04','zip','/content/gdrive/Shareddrives/YOLObee_drive/02_Code/YOLOv5/', base_dir='train_data')

'/content/gdrive/Shareddrives/YOLObee_drive/02_Code/train_data_04.zip'

Training
---

In [ ]:
%ls

datasets/    train_data/     train_data_03/  train_data1/  train_data_part1/  yolov5/
SciAugment/  train_data_02/  train_data_04/  train_data2/  train_data.zip


In [ ]:
%cd yolov5

/content/gdrive/Shareddrives/YOLObee_drive/02_Code/YOLOv5/yolov5


In [ ]:
#!git clone https://github.com/ultralytics/yolov5  # clone

#%cd yolov5

%pip install -r requirements.txt  # install

%cd ..

import torch

from yolov5 import utils

%cd yolov5
display = utils.notebook_init()  # checks

YOLOv5 🚀 v7.0-314-g60dde7f1 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 30.1/78.2 GB disk)


In [ ]:
%cd data
!wget https://github.com/martinschatz-cz/SciCount/raw/main/example_notebooks/WildlifeDetection/class1.yaml
%cd ..

/auto/brno2/home/jeanninc/YOLO/yolov5/data
--2024-06-13 14:31:57--  https://github.com/martinschatz-cz/SciCount/raw/main/example_notebooks/WildlifeDetection/class1.yaml
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/martinschatz-cz/SciCount/main/example_notebooks/WildlifeDetection/class1.yaml [following]
--2024-06-13 14:31:57--  https://raw.githubusercontent.com/martinschatz-cz/SciCount/main/example_notebooks/WildlifeDetection/class1.yaml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8002::154, 2606:50c0:8003::154, 2606:50c0:8000::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8002::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 234 [text/plain]
Saving to: ‘class1.yaml’

class1.yaml         100%[===================>]     

In [ ]:
!python -m torch.distributed.run --nproc_per_node 2 train.py --img 640 --batch 64 --epochs 800 --data class1.yaml --weights yolov5x.pt --cache --device 0,1

train: weights=yolov5x.pt, cfg=, data=class1.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=800, batch_size=64, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, bucket=, cache=ram, image_weights=False, device=0,1, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest, ndjson_console=False, ndjson_file=False
github: ⚠️ YOLOv5 is out of date by 5 commits. Use 'git pull' or 'git clone https://github.com/ultralytics/yolov5' to update.
YOLOv5 🚀 v7.0-321-g3742ab49 Python-3.10.12 torch-2.3.0a0+6ddf5cf85e.nv24.04 CUDA:0 (NVIDIA A40, 45403MiB)
                                                                             CUDA:1 (NVI

In [ ]:
# Train YOLOv5s on Smear images augmented train_data for 800 epochs
!python -m torch.distributed.run --nproc_per_node 2 train.py --img 640 --batch 64 --epochs 800 --data class1.yaml --weights /storage/brno2/home/jeanninc/YOLO/yolov5/runs/train/training_06_part1_x/weights/best.pt --cache --device 0,1

train: weights=/storage/brno2/home/jeanninc/YOLO/yolov5/runs/train/training_06_part1_x/weights/best.pt, cfg=, data=class1.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=800, batch_size=64, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, bucket=, cache=ram, image_weights=False, device=0,1, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest, ndjson_console=False, ndjson_file=False
remote: Enumerating objects: 34, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 34 (delta 15), reused 20 (delta 5), pack-reused 0
Unpacking objects: 100% (34/34), 40.16 KiB | 

Validation
---

In [ ]:
!python val.py --weights /storage/brno2/home/jeanninc/YOLO/yolov5/runs/train/training_06_part2_x/weights/best.pt --img 640 --data class1.yaml

val: data=/auto/brno2/home/jeanninc/YOLO/yolov5/data/class1.yaml, weights=['/storage/brno2/home/jeanninc/YOLO/yolov5/runs/train/training_06_part2_x/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-321-g3742ab49 Python-3.10.12 torch-2.3.0a0+6ddf5cf85e.nv24.04 CUDA:0 (NVIDIA A40, 45403MiB)

Fusing layers... 
Model summary: 322 layers, 86173414 parameters, 0 gradients, 203.8 GFLOPs
val: Scanning /auto/brno2/home/jeanninc/YOLO/train_data/labels/val.cache... 304 
                 Class     Images  Instances          P          R      mAP50   
                   all        304       1294      0.984      0.966      0.981      0.729
Speed: 0.1ms pre-process, 7.8ms inference, 3.6ms NMS per image at shape (32, 3, 640, 640)
Results 

Detection
---

An extract of the video NORM0001_b.MP4 called **video_2_bees_extract_1.mov**

In [ ]:
!python detect.py --weights /storage/brno2/home/jeanninc/YOLO/yolov5/runs/train/training_06_part2_x/weights/best.pt --img 640 --source '/storage/brno2/home/jeanninc/video/video_2_bees_extract_1.mov' --save-txt --save-conf


detect: weights=['/storage/brno2/home/jeanninc/YOLO/yolov5/runs/train/training_06_part2_x/weights/best.pt'], source=/storage/brno2/home/jeanninc/video/video_2_bees_extract_1.mov, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_csv=False, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-321-g3742ab49 Python-3.10.12 torch-2.3.0a0+6ddf5cf85e.nv24.04 CUDA:0 (NVIDIA A40, 45403MiB)

Fusing layers... 
Model summary: 322 layers, 86173414 parameters, 0 gradients, 203.8 GFLOPs
video 1/1 (1/265) /auto/brno2/home/jeanninc/video/video_2_bees_extract_1.mov: 384x640 6 bees, 36.0ms
video 1/1 (2/265) /auto/brno2/home/jeanninc/video/video_2_bees_extract_1.mov: 384x640 6 bees, 8.5ms
video 1/1 (3

An extract of the video NORM0001_b.MP4 called **video_2_bees_extract_2.mov**

In [ ]:
!python detect.py --weights /storage/brno2/home/jeanninc/YOLO/yolov5/runs/train/training_06_part2_x/weights/best.pt --img 640 --source '/storage/brno2/home/jeanninc/video/video_2_bees_extract_2.mov' --save-txt --save-conf


detect: weights=['/storage/brno2/home/jeanninc/YOLO/yolov5/runs/train/training_06_part2_x/weights/best.pt'], source=/storage/brno2/home/jeanninc/video/video_2_bees_extract_2.mov, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_csv=False, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-321-g3742ab49 Python-3.10.12 torch-2.3.0a0+6ddf5cf85e.nv24.04 CUDA:0 (NVIDIA A40, 45403MiB)

Fusing layers... 
Model summary: 322 layers, 86173414 parameters, 0 gradients, 203.8 GFLOPs
video 1/1 (1/215) /auto/brno2/home/jeanninc/video/video_2_bees_extract_2.mov: 384x640 8 bees, 34.1ms
video 1/1 (2/215) /auto/brno2/home/jeanninc/video/video_2_bees_extract_2.mov: 384x640 9 bees, 8.7ms
video 1/1 (3

An extract of the video NORM0001_b.MP4 called **video_2_bees_extract_3.mov**

In [ ]:
!python detect.py --weights /storage/brno2/home/jeanninc/YOLO/yolov5/runs/train/training_06_part2_x/weights/best.pt --img 640 --source '/storage/brno2/home/jeanninc/video/video_2_bees_extract_3.mov' --save-txt --save-conf


detect: weights=['/storage/brno2/home/jeanninc/YOLO/yolov5/runs/train/training_06_part2_x/weights/best.pt'], source=/storage/brno2/home/jeanninc/video/video_2_bees_extract_3.mov, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_csv=False, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-321-g3742ab49 Python-3.10.12 torch-2.3.0a0+6ddf5cf85e.nv24.04 CUDA:0 (NVIDIA A40, 45403MiB)

Fusing layers... 
Model summary: 322 layers, 86173414 parameters, 0 gradients, 203.8 GFLOPs
video 1/1 (1/361) /auto/brno2/home/jeanninc/video/video_2_bees_extract_3.mov: 384x640 7 bees, 33.8ms
video 1/1 (2/361) /auto/brno2/home/jeanninc/video/video_2_bees_extract_3.mov: 384x640 7 bees, 8.7ms
video 1/1 (3

detection of video_2_bees_extract_3.mov with a prediction threshold lower, at 0.25

In [ ]:
!python detect.py --weights /content/gdrive/Shareddrives/YOLOBee_weights/runs/train/training_06_part2_x/weights/best.pt --img 640 --conf 0.25 --source '/content/gdrive/Shareddrives/YOLObee_data/01_Data/video_2/video_2_bees_extract_1.mov' --save-txt --save-conf


detect: weights=['/content/gdrive/Shareddrives/YOLOBee_weights/runs/train/training_06_part2_x/weights/best.pt'], source=/content/gdrive/Shareddrives/YOLObee_data/01_Data/video_2/video_2_bees_extract_1.mov, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_csv=False, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-314-g60dde7f1 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 322 layers, 86173414 parameters, 0 gradients, 203.8 GFLOPs
WARNING ⚠️ NMS time limit 0.550s exceeded
video 1/1 (1/265) /content/gdrive/Shareddrives/YOLObee_data/01_Data/video_2/video_2_bees_extract_1.mov: 384x640 6 bees, 61.6ms
video 1/1 (2/265) /content/gdrive/